In [426]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics 
import math
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import r2_score

In [427]:
df=pd.read_csv("ENB2012_data.csv")
df

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [428]:
df.isnull().sum()

X1    0
X2    0
X3    0
X4    0
X5    0
X6    0
X7    0
X8    0
Y1    0
Y2    0
dtype: int64

### HİÇ NULL VERİ YOK
## Specifically:
* X1 Relative Compactness
* X2 Surface Area
* X3 Wall Area
* X4 Roof Area
* X5 Overall Height
* X6 Orientation
* X7 Glazing Area
* X8 Glazing Area Distribution
* y1 Heating Load
* y2 Cooling Load
## Sorunum:Genel yüksekliğin Isıtma yüküne etkisi

In [429]:
df.Y2.mean()
#df.X2.mean()

24.58776041666667

In [430]:
df.Y2-df.Y2.mean()

0     -3.25776
1     -3.25776
2     -3.25776
3     -3.25776
4      3.69224
        ...   
763   -3.18776
764   -7.70776
765   -7.47776
766   -7.97776
767   -8.55776
Name: Y2, Length: 768, dtype: float64

# MODEL SEÇİMİ



In [431]:
(df.Y2-df.Y2.mean()).pow(2).sum()

69415.78774791666

In [432]:
filt = (df.X5 <4)

In [433]:
df1=df[filt]

In [434]:
df2=df[~filt]

In [435]:
df1

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
24,0.74,686.0,245.0,220.5,3.5,2,0.0,0,6.07,10.90
25,0.74,686.0,245.0,220.5,3.5,3,0.0,0,6.05,11.19
26,0.74,686.0,245.0,220.5,3.5,4,0.0,0,6.01,10.94
27,0.74,686.0,245.0,220.5,3.5,5,0.0,0,6.04,11.17
28,0.71,710.5,269.5,220.5,3.5,2,0.0,0,6.37,11.27
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.5,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.5,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.5,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.5,3.5,4,0.4,5,16.48,16.61


In [436]:
df2

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
739,0.79,637.0,343.0,147.00,7.0,5,0.4,5,40.79,44.87
740,0.76,661.5,416.5,122.50,7.0,2,0.4,5,38.82,39.37
741,0.76,661.5,416.5,122.50,7.0,3,0.4,5,39.72,39.80
742,0.76,661.5,416.5,122.50,7.0,4,0.4,5,39.31,37.79


In [437]:
df1.Y2.mean()

16.071432291666667

In [438]:
df2.Y2.mean()

33.10408854166667

In [439]:
(df1.Y2-df1.Y2.mean()).pow(2).sum()+(df2.Y2-df2.Y2.mean()).pow(2).sum()

13714.402993229163

# BAŞKA BİR DEĞİŞKENE GÖRE KIRILIM YAPALIM X8'e göre
## X8= X8 Glazing Area Distribution

In [440]:
filt1=(df1.X8<=3)

In [441]:
df2=df1[filt1]

In [442]:
df3=df1[~filt1]

In [443]:
df2

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
24,0.74,686.0,245.0,220.5,3.5,2,0.0,0,6.07,10.90
25,0.74,686.0,245.0,220.5,3.5,3,0.0,0,6.05,11.19
26,0.74,686.0,245.0,220.5,3.5,4,0.0,0,6.01,10.94
27,0.74,686.0,245.0,220.5,3.5,5,0.0,0,6.04,11.17
28,0.71,710.5,269.5,220.5,3.5,2,0.0,0,6.37,11.27
...,...,...,...,...,...,...,...,...,...,...
667,0.64,784.0,343.0,220.5,3.5,5,0.4,3,18.46,21.53
668,0.62,808.5,367.5,220.5,3.5,2,0.4,3,16.47,16.90
669,0.62,808.5,367.5,220.5,3.5,3,0.4,3,16.35,17.14
670,0.62,808.5,367.5,220.5,3.5,4,0.4,3,16.55,16.56


In [444]:
df3


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
216,0.74,686.0,245.0,220.5,3.5,2,0.1,4,10.08,13.20
217,0.74,686.0,245.0,220.5,3.5,3,0.1,4,10.15,13.36
218,0.74,686.0,245.0,220.5,3.5,4,0.1,4,10.07,13.21
219,0.74,686.0,245.0,220.5,3.5,5,0.1,4,10.14,13.53
220,0.71,710.5,269.5,220.5,3.5,2,0.1,4,10.66,13.67
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.5,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.5,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.5,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.5,3.5,4,0.4,5,16.48,16.61


In [445]:
(df1.Y2-df1.Y2.mean()).pow(2).sum()+(df2.Y2-df2.Y2.mean()).pow(2).sum()+(df3.Y2-df3.Y2.mean()).pow(2).sum()

4482.409791822916

### HATA DEĞERİM YARILANDI TEKRAR BİR DEĞİŞKENE GÖRE FİLTRELEME YAPALIM
#### df3 veri setini tekrar filtreleme yapalım bu sefer X7 değişkenine göre

In [446]:
filt2=(df3.X7<=0.25)

In [447]:
df4=df3[filt2]

In [448]:
df5=df3[~filt2]

In [449]:
df4.Y2.mean()

15.543125000000002

In [450]:
df5.Y2.mean()

17.63375

In [451]:
(df1.Y2-df1.Y2.mean()).pow(2).sum()+(df2.Y2-df2.Y2.mean()).pow(2).sum()+(df4.Y2-df4.Y2.mean()).pow(2).sum()+(df5.Y2-df5.Y2.mean()).pow(2).sum()

4342.546979322917

### RMSE'

In [452]:
df["Y2"]=np.log(df.Y2)

In [453]:
m=RandomForestRegressor(n_estimators=26,bootstrap=True,n_jobs=-1)
m.fit(df.drop('Y2',axis=1),df.Y2)
m.score(df.drop('Y2',axis=1),df.Y2)

0.9981831372998019

In [454]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

In [455]:
rmse(m.predict(df.drop('Y2',axis=1)),df.Y2)

0.01672939302504074

In [456]:
def print_score(m):
    print(f"RMSE of valid {rmse(m.predict(df.drop('Y2',axis=1)),df.Y2)}")
    print(f"R^2 of valid {m.score(df.drop('Y2',axis=1),df.Y2)}")

In [459]:
print_score(m)

RMSE of valid 0.01672939302504075
R^2 of valid 0.9981831372998019
